In [1]:
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from scipy.optimize import curve_fit,Bounds
from scipy.integrate import simps
import zfit_physics as zphysics
import zfit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import mplhep
import uncertainties 
from uncertainties import ufloat
import tensorflow as tf

plt.style.use('belle2')
#%jsroot on


%run /afs/desy.de/user/j/jipatel/MT/MC_fit/Files_After_Acceptance/Signal_Background_thesis_final/Function.ipynb

2024-09-03 12:49:23.472852: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 12:49:23.503526: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 12:49:23.504440: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 12:49:24.324678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warnin

## 200

In [3]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 200
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"],  'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"], 'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"],  'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"],  'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)


    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_200.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_200.csv'")


Running iteration 1/300...
200
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6431                 188.879  -     16   +     17       False
background_yield_0.6431             391.145  -     21   +     22       False
frac_Mbc_bkg1_0.6431               0.853318  -  0.066   +  0.063       False
frac_deltaE_bkg1_0.6431            0.268892  -   0.29   +    0.3       False
True
Iteration 1 - Signal Yield: 188.87920514413486 ± 16.27521504693627
Running iteration 2/300...
200
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6084                 182.405  -     16   +     17       False
background_yield_0.6084              397.68  -     22   +     22       False
frac_Mbc_bkg1_0.6084                0.87408  -  0.064   +  0.061       False
frac_deltaE

## 150

In [7]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 150
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_150.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_150.csv'")


Running iteration 1/300...
150
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7449                 141.648  -     14   +     15       False
background_yield_0.7449             388.557  -     21   +     21       False
frac_Mbc_bkg1_0.7449               0.844281  -  0.065   +  0.064       False
frac_deltaE_bkg1_0.7449            0.309313  -    0.3   +    0.3       False
True
Iteration 1 - Signal Yield: 141.64754508113262 ± 14.484287861209772
Running iteration 2/300...
150
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3719                 132.166  -     14   +     15       False
background_yield_0.3719             397.895  -     21   +     22       False
frac_Mbc_bkg1_0.3719               0.875535  -  0.063   +  0.061       False
frac_delta

## 100

In [8]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 100
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_100.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_100.csv'")


Running iteration 1/300...
100
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8532                 93.5311  -     12   +     13       False
background_yield_0.8532              386.27  -     20   +     22       False
frac_Mbc_bkg1_0.8532               0.838081  -  0.065   +  0.063       False
frac_deltaE_bkg1_0.8532             0.26801  -   0.29   +    0.3       False
True
Iteration 1 - Signal Yield: 93.5311104299182 ± 12.257123811879087
Running iteration 2/300...
100
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4871                 86.7542  -     12   +     13       False
background_yield_0.4871             393.263  -     21   +     22       False
frac_Mbc_bkg1_0.4871               0.862668  -  0.065   +   0.06       False
frac_deltaE_

## 75

In [9]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 75
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_75.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_75.csv'")


Running iteration 1/300...
75
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8483                 70.9686  -     10   +     12       False
background_yield_0.8483             383.446  -     20   +     22       False
frac_Mbc_bkg1_0.8483                0.83357  -  0.066   +  0.063       False
frac_deltaE_bkg1_0.8483            0.273633  -   0.29   +   0.29       False
True
Iteration 1 - Signal Yield: 70.96861301652147 ± 11.10248686090142
Running iteration 2/300...
75
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0584                 61.1372  -     11   +     11       False
background_yield_0.0584             393.834  -     21   +     22       False
frac_Mbc_bkg1_0.0584               0.861983  -  0.064   +  0.061       False
frac_deltaE_bk

## 50

In [10]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 50
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_50.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_50.csv'")


Running iteration 1/300...
50
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6552                 45.5469  -    9.3   +    9.8       False
background_yield_0.6552             384.397  -     20   +     21       False
frac_Mbc_bkg1_0.6552               0.834947  -  0.065   +  0.063       False
frac_deltaE_bkg1_0.6552            0.270182  -   0.29   +    0.3       False
True
Iteration 1 - Signal Yield: 45.5469154374551 ± 9.565695229350776
Running iteration 2/300...
50
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9757                 34.0518  -    8.8   +    9.5       False
background_yield_0.9757             395.904  -     21   +     22       False
frac_Mbc_bkg1_0.9757               0.867052  -  0.063   +   0.06       False
frac_deltaE_bkg

frac_Mbc_bkg1_0.6665               0.310272  -  0.036   +  0.036       False
frac_deltaE_bkg1_0.6665            0.055113  -   0.13   +   0.13       False
True
Iteration 96 - Signal Yield: 2433.3246713255635 ± 53.85116435998698
Running iteration 97/100...
2381.8491017785136
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3964                 2471.24  -     54   +     55       False
background_yield_0.3964             1980.13  -     49   +     50       False
frac_Mbc_bkg1_0.3964               0.290972  -  0.037   +  0.036       False
frac_deltaE_bkg1_0.3964           0.0287554  -   0.13   +   0.13       False
True
Iteration 97 - Signal Yield: 2471.2435562409755 ± 54.14350429115731
Running iteration 98/100...
2391.0938153673433
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  ------

## 25

In [11]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 25
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_25.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_25.csv'")


Running iteration 1/300...
25
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6753                  20.865  -    7.2   +    8.1       False
background_yield_0.6753             383.766  -     20   +     21       False
frac_Mbc_bkg1_0.6753               0.835927  -  0.065   +  0.062       False
frac_deltaE_bkg1_0.6753            0.177128  -   0.29   +   0.31       False
True
Iteration 1 - Signal Yield: 20.865000576341636 ± 7.647053047144316
Running iteration 2/300...
25
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0312                 9.43516  -    6.2   +    7.5       False
background_yield_0.0312             395.113  -     20   +     21       False
frac_Mbc_bkg1_0.0312               0.863828  -  0.062   +   0.06       False
frac_deltaE_b

frac_Mbc_bkg1_0.6376                0.31163  -  0.035   +  0.034       False
frac_deltaE_bkg1_0.6376            0.140314  -   0.13   +   0.13       False
True
Iteration 96 - Signal Yield: 1494.407415348163 ± 43.21497133215972
Running iteration 97/100...
1413.3656510900826
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5005                 1460.13  -     43   +     43       False
background_yield_0.5005             2022.62  -     49   +     50       False
frac_Mbc_bkg1_0.5005               0.342609  -  0.034   +  0.034       False
frac_deltaE_bkg1_0.5005            0.079136  -   0.12   +   0.13       False
True
Iteration 97 - Signal Yield: 1460.1322637061808 ± 43.15133797622084
Running iteration 98/100...
1436.8258689399797
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  -------

## 10

In [12]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 10
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_10.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_10.csv'")


Running iteration 1/300...
10
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1321                 5.70404  -    5.5   +    6.4       False
background_yield_0.1321             384.294  -     20   +     21       False
frac_Mbc_bkg1_0.1321               0.834932  -  0.064   +  0.062       False
frac_deltaE_bkg1_0.1321            0.170372  -   0.29   +    0.3       False
True
Iteration 1 - Signal Yield: 5.704042056685983 ± 5.955522216907967
Running iteration 2/300...
10
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6444                 2.71646  -    5.1   +    5.9       False
background_yield_0.6444             387.194  -     20   +     21       False
frac_Mbc_bkg1_0.6444               0.843661  -  0.063   +  0.061       False
frac_deltaE_bk

## 5

In [13]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 5
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_5.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_5.csv'")


Running iteration 1/300...
5
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8986                 2.53573  -    4.8   +    5.7       False
background_yield_0.8986             382.499  -     20   +     21       False
frac_Mbc_bkg1_0.8986               0.829461  -  0.064   +  0.062       False
frac_deltaE_bkg1_0.8986            0.153827  -   0.29   +    0.3       False
True
Iteration 1 - Signal Yield: 2.5357341250578154 ± 5.290065440355177
Running iteration 2/300...
5
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6615                0.727208  -    4.7   +    5.5       False
background_yield_0.6615             385.048  -     21   +     20       False
frac_Mbc_bkg1_0.6615               0.835265  -  0.064   +  0.061       False
frac_deltaE_bkg

frac_Mbc_bkg1_0.7276               0.354897  -  0.032   +  0.032       False
frac_deltaE_bkg1_0.7276            0.168362  -   0.12   +   0.12       False
True
Iteration 96 - Signal Yield: 517.791784443369 ± 28.100793762199558
Running iteration 97/100...
471.5720563185149
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1195                 520.234  -     28   +     28       False
background_yield_0.1195             2020.95  -     47   +     48       False
frac_Mbc_bkg1_0.1195               0.348573  -  0.032   +  0.032       False
frac_deltaE_bkg1_0.1195            0.193819  -   0.12   +   0.13       False
True
Iteration 97 - Signal Yield: 520.2343523497123 ± 28.04466287834341
Running iteration 98/100...
480.2083299603238
2069.535642151438
name                       value  (rounded)               errors    at limit
-----------------------  ----------

## 2

In [15]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 2
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_2.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_2.csv'")


Running iteration 1/300...
2
380
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.181              -0.0754711  -    4.4   +    5.3       False
background_yield_0.181             382.101  -     20   +     20       False
frac_Mbc_bkg1_0.181               0.827829  -  0.064   +  0.062       False
frac_deltaE_bkg1_0.181            0.167536  -   0.29   +    0.3       False
True
Iteration 1 - Signal Yield: -0.07547108027468287 ± 4.8565371280435095
Running iteration 2/300...
2
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3023                -1.51397  -    4.3   +    5.1       False
background_yield_0.3023             383.644  -     20   +     20       False
frac_Mbc_bkg1_0.3023               0.831773  -  0.063   +  0.062       False
frac_deltaE_bkg1_0

## 0

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_B0pip,file_bkg_ele_B0pip, 'ele', '511', 'pip', 'ee', 3, 'rec511', 'weights', 0
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_ee_signal_yields_and_errors_0.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_ee_signal_yields_and_errors_0.csv'")


Running iteration 1/300...
0
380


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3398                -2.46069  -      4   +      5       False
background_yield_0.3398             382.449  -     20   +     20       False
frac_Mbc_bkg1_0.3398               0.828502  -  0.064   +  0.062       False
frac_deltaE_bkg1_0.3398            0.167259  -   0.29   +    0.3       False
True
Iteration 1 - Signal Yield: -2.4606859034644537 ± 4.4839458521960704
Running iteration 2/300...
0
380
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6526                -2.46069  -      4   +      5       False
background_yield_0.6526             382.449  -     20   +     20       False
frac_Mbc_bkg1_0.6526               0.828502  -  0.064   +  0.062       False
frac_deltaE_bkg1_0.6526            0.167259  -